# Laboratório 05: Expansão de Consultas

Antes de responder as perguntas, precisamos dos documentos que serão extraídos do CSV, além das 5 consultas de um termo escolhidas na questão 2.1 do Laboratório 04 e dos índices obtidos anteriormente.



*  **Importação das extensões necessárias** 

In [1]:
import csv
import pandas as pd
import numpy as np
import nltk
import re
import collections
import math
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('rslp')
import heapq
import time
import bisect

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.



*   **Leitura do CSV e criação do ranking**

In [0]:
colecao = pd.read_csv('https://raw.githubusercontent.com/LDVictor/ri_lab_01/master/output/results.csv')
colecao = colecao.astype(str)
documentos = colecao['texto']

In [0]:
tokenizador = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
indices = {}
n = 0

for texto in documentos:
  palavras = [palavra for palavra in tokenizador.tokenize(texto.lower())
           if not bool(re.search(r'\d', palavra))
           and palavra not in stopwords and len(palavra) >= 3]  
  n += 1
  for t in palavras:
    if t not in indices.keys():
      indices[t] = []
    indices[t].append(n)
    
for elemento in indices.items():
  d = dict(collections.Counter(elemento[1]))
  indices[elemento[0]] = list(d.items())



*  **Definição das 5 consultas**

In [0]:
consultas = ["presidente", "educação", "privatização", "empresa", "bolsonaro"]

# Questão 1

**Calcule as top-10 palavras mais associadas a cada uma dessas 5 palavras de acordo com as 4 métricas que vimos na aula. Você deve produzir uma tabela similar à tabela 6.3 do capítulo 6 do livro texto (pág. 204).**


Primeiramente, iremos definir as funções para calcular cada uma das métricas: Dice's Coefficient (Dice), Mutual Information Measure (MIM), Expected Mutual Information Measure (EMIM) e Pearson's Chi-squared (χ²).

In [0]:
def valorMIM(a, b, ranking):
  na = calculaNx(a, ranking)
  nb = calculaNx(b, ranking)
  nab = calculaNab(a, b, ranking)
  if (na * nb) != 0:
    mim = nab / (na * nb)
  else:
    mim = 0
  return (mim, b)
  
def valorEMIM(a, b, ranking, n):
  na = calculaNx(a, ranking)
  nb = calculaNx(b, ranking)
  nab = calculaNab(a, b, ranking)
  div = (na * nb)
  if div != 0:
    exp = n * (nab / div)
  else:
    exp = 0
  if exp != 0:
    emim = nab * np.log10(exp)
  else:
    emim = 0
  return (na, b)
  
def valorChiSquared(a, b, ranking, n):
  na = calculaNx(a, ranking)
  nb = calculaNx(b, ranking)
  nab = calculaNab(a, b, ranking)
  if (na * nb) != 0:
    chi_squared = ((nab - (1 / n) * na * nb) ** 2 / (na * nb))
  else:
    chi_squared = 0
  return (chi_squared, b)
  
def valorDice(a, b, ranking):
  na = calculaNx(a, ranking)
  nb = calculaNx(b, ranking)
  nab = calculaNab(a, b, ranking)
  dice = (nab / (na + nb))
  return (dice, b)

De maneira análoga, também precisamos definir como calcular os valores de Na, Nb e Nab para usar nas métricas definidas.

In [0]:
def calculaNx(palavraX, ranking):
  return len(ranking[palavraX])

def calculaNab(palavraA, palavraB, ranking):
  documentosA = [documento for documento, pontos in ranking[palavraA]]
  documentosB = [documento for documento, pontos in ranking[palavraB]]
  n = 0
  
  for documento in documentosA:
    if documento in documentosB:
      n += 1
      
  return n

Com as funções já criadas, podemos agora percorrer os índices para cada consulta, armazenando os resultados dos cálculos de cada métrica. Para isso, criaremos uma função que, dada a consulta, retornará uma tabela com as 10 palavras mais associadas a cada consulta de acordo com as métricas que definimos acima.

In [0]:
def criaTabela(consulta, ranking, n):
  dice = []
  mim = []
  emim = []
  chi_squared = []
  
  for palavra in ranking:
    if palavra != consulta:
      bisect.insort(dice, valorDice(consulta, palavra, ranking))
      bisect.insort(mim, valorMIM(consulta, palavra, ranking))
      bisect.insort(emim, valorEMIM(consulta, palavra, ranking, n))
      bisect.insort(chi_squared, valorChiSquared(consulta, palavra, ranking, n))

  dice.reverse()    
  mim.reverse()
  emim.reverse()
  chi_squared.reverse()
  
  tabela_metricas = pd.DataFrame(columns=['MIM', 'EMIM', 'χ²', 'Dice'])

  tabela_metricas['MIM'] = [palavra[1] for palavra in mim]
  tabela_metricas['EMIM'] = [palavra[1] for palavra in emim]
  tabela_metricas['χ²'] = [palavra[1] for palavra in chi_squared]
  tabela_metricas['Dice'] = [palavra[1] for palavra in dice]
  
  tabela_metricas.index += 1
  
  return tabela_metricas.head(10)

Por fim, chamaremos a função para cada consulta do vetor.




1.   Consulta "presidente"

In [8]:
n = colecao.texto.count()
tabela_consulta_1 = criaTabela(consultas[0], indices, n)
tabela_consulta_1

,MIM,EMIM,χ²,Dice
1,útil,útil,presidência,governo
2,úteis,úteis,feira,ser
3,únicos,únicos,forças,brasil
4,ótimos,único,armadas,sobre
5,ótimo,únicas,trump,país
6,ótica,única,posição,bolsonaro
7,ódios,últimos,brasil,feira
8,óculos,último,parte,ainda
9,íntima,últimas,venezuela,disse
10,índole,última,casa,segundo



2.   Consulta "educação"



In [9]:
tabela_consulta_2 = criaTabela(consultas[1], indices, n)
tabela_consulta_2

,MIM,EMIM,χ²,Dice
1,íntimas,útil,constrangimento,sociais
2,íntima,úteis,substancial,instituto
3,ânima,únicos,holocausto,redes
4,âncora,único,instituto,ricardo
5,ávila,únicas,nacionalista,violência
6,álibis,única,durou,culpa
7,zumba,últimos,comentários,alunos
8,zelar,último,ricardo,constrangimento
9,zeidan,últimas,sociais,faz
10,ynaê,última,sofrimento,escolas



3.   Consulta "privatização"

In [10]:
tabela_consulta_3 = criaTabela(consultas[2], indices, n)
tabela_consulta_3

,MIM,EMIM,χ²,Dice
1,ámbito,útil,rentabilidade,rentabilidade
2,zelar,úteis,juros,juros
3,véu,únicos,externo,externo
4,voos,único,melhora,melhora
5,vencidas,únicas,consistente,consistente
6,venciam,única,beneficia,beneficia
7,vastas,últimos,aprofundamento,aprofundamento
8,variações,último,impostos,impostos
9,variação,últimas,ámbito,sustentar
10,turbulências,última,zelar,pressões



4.   Consulta "empresa"

In [11]:
tabela_consulta_4 = criaTabela(consultas[3], indices, n)
tabela_consulta_4

,MIM,EMIM,χ²,Dice
1,ânimo,útil,desculpas,brasília
2,ânima,úteis,irregularidades,empresas
3,zoneamento,únicos,coronel,reais
4,zimmermann,único,brasília,coronel
5,yousseff,únicas,sócio,área
6,younger,única,rita,duas
7,xingam,últimos,morre,pagamento
8,xerife,último,nicas,corrupção
9,werner,últimas,irregularidade,lima
10,wanzhou,última,delatores,risco




5.   Consulta "bolsonaro"


In [12]:
tabela_consulta_5 = criaTabela(consultas[4], indices, n)
tabela_consulta_5

,MIM,EMIM,χ²,Dice
1,útil,útil,jair,brasil
2,úteis,úteis,brasil,jair
3,únicos,únicos,psl,presidente
4,ótimos,único,deputado,governo
5,íntima,únicas,campanha,ser
6,índole,única,filhos,ainda
7,ímpetos,últimos,presidência,disse
8,ênfase,último,capitão,sobre
9,étnicos,últimas,araújo,ter
10,étnicas,última,eleição,diz



**Qual métrica você acha que obteve os melhores resultados? Por que?**

Após a análise de todos os resultados das tabelas acima, a métrica do Coeficiente de Dice aparentou ter os melhores resultados para as cinco consultas definidas pelo fato de ter sido a mais estável e confiável na variedade de palavras usadas. Unindo os bons resultados a sua baixa complexidade comparada as outras métricas, podemos definir o Coeficiente de Dice como a melhor métrica para o caso testado.

# Questão 2

**De acordo com a métrica que deu os melhores resultados na sua opinião, execute agora cada consulta (usando a abordagem documento- ou termo-por-vez)  expandido-a com: os top-3, top-5 e top-10 documentos.**

Primeiramente, precisamos da função que segue uma abordagem documento-por-vez ou termo-por-vez. Para essa questão, iremos escolher a abordagem documento-por-vez implementada na última atividade de laboratório.

In [0]:
def documentoPorVez(consulta, indices, k):
    listas_invertidas = []
    r = []
    for palavra in consulta.split(" "):
        if palavra in indices.keys():
            listas_invertidas.append(indices[palavra])
    for documento in range(1, len(documentos)+1):
        sd = 0
        for lista_invertida in listas_invertidas:
            for i in lista_invertida:
                if (i[0] == documento): 
                    sd += i[1]
                    break
        if (sd != 0):
          heapq.heappush(r, (sd, documento))
    return heapq.nlargest(k, r)

Agora, montaremos o ranking das palavras considerando apenas os top-k documentos para a palavra da consulta. E então, fazendo uso do Coeficiente de Dice, expandiremos a consulta a partir desse ranking para todas as palavras no ranking inicial.

In [0]:
def expansaoConsulta(consulta, k):
  
  expansao_documentos = [documento for pontos, documento in documentoPorVez(consulta, indices, k)]
  ranking_expandido = {} 
  
  for palavra in indices:
    ranking_expandido[palavra] = []
    for pontos_documento in indices[palavra]:
      if pontos_documento[0] in expansao_documentos:
        ranking_expandido[palavra].append(pontos_documento)
        
  return ranking_expandido


*   **Documentos Top 3**


In [0]:
k = 3
documentos_ranking_1 = expansaoConsulta(consultas[0], k)
documentos_ranking_2 = expansaoConsulta(consultas[1], k)
documentos_ranking_3 = expansaoConsulta(consultas[2], k)
documentos_ranking_4 = expansaoConsulta(consultas[3], k)
documentos_ranking_5 = expansaoConsulta(consultas[4], k)


tabela_top_1 = criaTabela(consultas[0], documentos_ranking_1, k)
tabela_top_2 = criaTabela(consultas[1], documentos_ranking_2, k)
tabela_top_3 = criaTabela(consultas[2], documentos_ranking_3, k)
tabela_top_4 = criaTabela(consultas[3], documentos_ranking_4, k)
tabela_top_5 = criaTabela(consultas[4], documentos_ranking_5, k)


1.   Top-10 palavras associadas aos Top-3 documentos da consulta "presidente"

In [16]:
tabela_dice_1 = pd.DataFrame()
tabela_dice_1['Dice sem Documento por Vez'] = tabela_consulta_1['Dice']
tabela_dice_1['Dice com Documento por Vez'] = tabela_top_1['Dice']
tabela_dice_1

,Dice sem Documento por Vez,Dice com Documento por Vez
1,governo,pressionar
2,ser,povo
3,brasil,passado
4,sobre,outros
5,país,nacional
6,bolsonaro,mensagem
7,feira,jair
8,ainda,governo
9,disse,fez
10,segundo,disse



2.   Top-10 palavras associadas aos Top-3 documentos da consulta "educação"


In [17]:
tabela_dice_2 = pd.DataFrame()
tabela_dice_2['Dice sem Documento por Vez'] = tabela_consulta_2['Dice']
tabela_dice_2['Dice com Documento por Vez'] = tabela_top_2['Dice']
tabela_dice_2

,Dice sem Documento por Vez,Dice com Documento por Vez
1,sociais,único
2,instituto,vida
3,redes,uso
4,ricardo,trabalho
5,violência,todas
6,culpa,ter
7,alunos,situação
8,constrangimento,sido
9,faz,ser
10,escolas,segundo



3.   Top-10 palavras associadas aos Top-3 documentos da consulta "privatização"

In [18]:
tabela_dice_3 = pd.DataFrame()
tabela_dice_3['Dice sem Documento por Vez'] = tabela_consulta_3['Dice']
tabela_dice_3['Dice com Documento por Vez'] = tabela_top_3['Dice']
tabela_dice_3

,Dice sem Documento por Vez,Dice com Documento por Vez
1,rentabilidade,empresas
2,juros,vida
3,externo,transição
4,melhora,trabalhador
5,consistente,taxa
6,beneficia,sustentar
7,aprofundamento,social
8,impostos,sobre
9,sustentar,sistema
10,pressões,setor



4.   Top-10 palavras associadas aos Top-3 documentos da consulta "empresa"

In [19]:
tabela_dice_4 = pd.DataFrame()
tabela_dice_4['Dice sem Documento por Vez'] = tabela_consulta_4['Dice']
tabela_dice_4['Dice com Documento por Vez'] = tabela_top_4['Dice']
tabela_dice_4

,Dice sem Documento por Vez,Dice com Documento por Vez
1,brasília,valor
2,empresas,vai
3,reais,setor
4,coronel,ser
5,área,presidente
6,duas,pediu
7,pagamento,parte
8,corrupção,nota
9,lima,feira
10,risco,duas


5.   Top-10 palavras associadas aos Top-3 documentos da consulta "bolsonaro"

In [20]:
tabela_dice_5 = pd.DataFrame()
tabela_dice_5['Dice sem Documento por Vez'] = tabela_consulta_5['Dice']
tabela_dice_5['Dice com Documento por Vez'] = tabela_top_5['Dice']
tabela_dice_5

,Dice sem Documento por Vez,Dice com Documento por Vez
1,brasil,vai
2,jair,têm
3,presidente,tom
4,governo,todo
5,ser,tanto
6,ainda,sobre
7,disse,sido
8,sobre,ser
9,ter,sempre
10,diz,segundo




*   **Documentos Top 5**


In [0]:
k = 5
documentos_ranking_1 = expansaoConsulta(consultas[0], k)
documentos_ranking_2 = expansaoConsulta(consultas[1], k)
documentos_ranking_3 = expansaoConsulta(consultas[2], k)
documentos_ranking_4 = expansaoConsulta(consultas[3], k)
documentos_ranking_5 = expansaoConsulta(consultas[4], k)


tabela_top_1 = criaTabela(consultas[0], documentos_ranking_1, k)
tabela_top_2 = criaTabela(consultas[1], documentos_ranking_2, k)
tabela_top_3 = criaTabela(consultas[2], documentos_ranking_3, k)
tabela_top_4 = criaTabela(consultas[3], documentos_ranking_4, k)
tabela_top_5 = criaTabela(consultas[4], documentos_ranking_5, k)


1.   Top-10 palavras associadas aos Top-5 documentos da consulta "presidente"

In [22]:
tabela_dice_1 = pd.DataFrame()
tabela_dice_1['Dice sem Documento por Vez'] = tabela_consulta_1['Dice']
tabela_dice_1['Dice com Documento por Vez'] = tabela_top_1['Dice']
tabela_dice_1

,Dice sem Documento por Vez,Dice com Documento por Vez
1,governo,outros
2,ser,governo
3,brasil,fez
4,sobre,disse
5,país,bolsonaro
6,bolsonaro,ainda
7,feira,venezuela
8,ainda,ter
9,disse,presidência
10,segundo,povo



2.   Top-10 palavras associadas aos Top-5 documentos da consulta "educação"

In [23]:
tabela_dice_2 = pd.DataFrame()
tabela_dice_2['Dice sem Documento por Vez'] = tabela_consulta_2['Dice']
tabela_dice_2['Dice com Documento por Vez'] = tabela_top_2['Dice']
tabela_dice_2

,Dice sem Documento por Vez,Dice com Documento por Vez
1,sociais,trabalho
2,instituto,todas
3,redes,ter
4,ricardo,segundo
5,violência,ricardo
6,culpa,ministério
7,alunos,ministro
8,constrangimento,menos
9,faz,maior
10,escolas,família



3.   Top-10 palavras associadas aos Top-5 documentos da consulta "privatização"



In [24]:
tabela_dice_3 = pd.DataFrame()
tabela_dice_3['Dice sem Documento por Vez'] = tabela_consulta_3['Dice']
tabela_dice_3['Dice com Documento por Vez'] = tabela_top_3['Dice']
tabela_dice_3

,Dice sem Documento por Vez,Dice com Documento por Vez
1,rentabilidade,ser
2,juros,reforma
3,externo,governo
4,melhora,social
5,consistente,setor
6,beneficia,serviços
7,aprofundamento,segundo
8,impostos,redução
9,sustentar,público
10,pressões,outro


4.   Top-10 palavras associadas aos Top-5 documentos da consulta "empresa"


In [25]:
tabela_dice_4 = pd.DataFrame()
tabela_dice_4['Dice sem Documento por Vez'] = tabela_consulta_4['Dice']
tabela_dice_4['Dice com Documento por Vez'] = tabela_top_4['Dice']
tabela_dice_4

,Dice sem Documento por Vez,Dice com Documento por Vez
1,brasília,setor
2,empresas,ser
3,reais,presidente
4,coronel,parte
5,área,duas
6,duas,após
7,pagamento,valores
8,corrupção,valor
9,lima,sob
10,risco,segundo


5.   Top-10 palavras associadas aos Top-5 documentos da consulta "bolsonaro"


In [26]:
tabela_dice_5 = pd.DataFrame()
tabela_dice_5['Dice sem Documento por Vez'] = tabela_consulta_5['Dice']
tabela_dice_5['Dice com Documento por Vez'] = tabela_top_5['Dice']
tabela_dice_5

,Dice sem Documento por Vez,Dice com Documento por Vez
1,brasil,vai
2,jair,sobre
3,presidente,ser
4,governo,segundo
5,ser,presidente
6,ainda,nada
7,disse,flávio
8,sobre,fazer
9,ter,disse
10,diz,deputado




*   **Documentos Top 10**

In [0]:
k = 10
documentos_ranking_1 = expansaoConsulta(consultas[0], k)
documentos_ranking_2 = expansaoConsulta(consultas[1], k)
documentos_ranking_3 = expansaoConsulta(consultas[2], k)
documentos_ranking_4 = expansaoConsulta(consultas[3], k)
documentos_ranking_5 = expansaoConsulta(consultas[4], k)


tabela_top_1 = criaTabela(consultas[0], documentos_ranking_1, k)
tabela_top_2 = criaTabela(consultas[1], documentos_ranking_2, k)
tabela_top_3 = criaTabela(consultas[2], documentos_ranking_3, k)
tabela_top_4 = criaTabela(consultas[3], documentos_ranking_4, k)
tabela_top_5 = criaTabela(consultas[4], documentos_ranking_5, k)


1.   Top-10 palavras associadas aos Top-10 documentos da consulta "presidente"


In [28]:
tabela_dice_1 = pd.DataFrame()
tabela_dice_1['Dice sem Documento por Vez'] = tabela_consulta_1['Dice']
tabela_dice_1['Dice com Documento por Vez'] = tabela_top_1['Dice']
tabela_dice_1

,Dice sem Documento por Vez,Dice com Documento por Vez
1,governo,bolsonaro
2,ser,nacional
3,brasil,brasil
4,sobre,ainda
5,país,parte
6,bolsonaro,outros
7,feira,grande
8,ainda,governo
9,disse,fez
10,segundo,disse



2.   Top-10 palavras associadas aos Top-10 documentos da consulta "educação"

In [29]:
tabela_dice_2 = pd.DataFrame()
tabela_dice_2['Dice sem Documento por Vez'] = tabela_consulta_2['Dice']
tabela_dice_2['Dice com Documento por Vez'] = tabela_top_2['Dice']
tabela_dice_2

,Dice sem Documento por Vez,Dice com Documento por Vez
1,sociais,ser
2,instituto,segundo
3,redes,menos
4,ricardo,bolsonaro
5,violência,ano
6,culpa,ainda
7,alunos,violência
8,constrangimento,ter
9,faz,sociais
10,escolas,onde


3.   Top-10 palavras associadas aos Top-10 documentos da consulta "privatização"

In [30]:
tabela_dice_3 = pd.DataFrame()
tabela_dice_3['Dice sem Documento por Vez'] = tabela_consulta_3['Dice']
tabela_dice_3['Dice com Documento por Vez'] = tabela_top_3['Dice']
tabela_dice_3

,Dice sem Documento por Vez,Dice com Documento por Vez
1,rentabilidade,ser
2,juros,reforma
3,externo,governo
4,melhora,social
5,consistente,setor
6,beneficia,serviços
7,aprofundamento,segundo
8,impostos,redução
9,sustentar,público
10,pressões,outro


4.   Top-10 palavras associadas aos Top-10 documentos da consulta "empresa"

In [31]:
tabela_dice_4 = pd.DataFrame()
tabela_dice_4['Dice sem Documento por Vez'] = tabela_consulta_4['Dice']
tabela_dice_4['Dice com Documento por Vez'] = tabela_top_4['Dice']
tabela_dice_4

,Dice sem Documento por Vez,Dice com Documento por Vez
1,brasília,segundo
2,empresas,presidente
3,reais,parte
4,coronel,caso
5,área,ainda
6,duas,ser
7,pagamento,grupo
8,corrupção,empresas
9,lima,duas
10,risco,dia


5.   Top-10 palavras associadas aos Top-10 documentos da consulta "bolsonaro"

In [32]:
tabela_dice_5 = pd.DataFrame()
tabela_dice_5['Dice sem Documento por Vez'] = tabela_consulta_5['Dice']
tabela_dice_5['Dice com Documento por Vez'] = tabela_top_5['Dice']
tabela_dice_5

,Dice sem Documento por Vez,Dice com Documento por Vez
1,brasil,ser
2,jair,presidente
3,presidente,brasil
4,governo,agora
5,ser,vai
6,ainda,ter
7,disse,sobre
8,sobre,paulo
9,ter,disse
10,diz,contra


**O que acontece com a precisão dos resultados em cada caso? Aumenta ou diminui? Justifique bem sua resposta.**

De acordo com os resultados, percebe-se que para o top-3 algumas consultas acabam ficando irrelevantes, fazendo com que a tabela mostre palavras sem significância no contexto, mesmo desconsiderando as stopwords, fazendo com que a qualidade dos resultados não seja satisfatória. Entretanto, com relação aos top-5 e top-10, apresentam-se casos mais satisfatórios, podendo assim ser justificados pelo fato de considerar documentos que realmente tem sentido dentro do contexto da consulta.